In [81]:
%pip install pandas numpy matplotlib seaborn statsmodels requests openpyxl yfinance kaggle python-dotenv

5914.34s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [82]:
%pip install /home/pinagm/beaapi-0.0.2-py3-none-any.whl

5923.20s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Processing /home/pinagm/beaapi-0.0.2-py3-none-any.whl
beaapi is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Note: you may need to restart the kernel to use updated packages.


In [85]:
import pkg_resources
from subprocess import call

packages = [dist.project_name for dist in pkg_resources.working_set]
call("pip install --upgrade " + ' '.join(packages), shell=True)

/tmp/ipykernel_3412/638084235.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


  Using cached Deprecated-1.2.18-py2.py3-none-any.whl.metadata (5.7 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached Mako-1.3.9-py3-none-any.whl.metadata (2.9 kB)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached alembic-1.14.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached beautifulsoup4-4.13.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached executing-2.2.0-py2.py3-none-any.whl.metadata (8.9 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached google_auth-2.38.0-py2.py3-none-any.whl.metadata (4.8 kB)
  Using cached graphql_core-3.2.6-py3-none-any.whl.metadata (11 kB)
  Using cached gunicorn-23.0.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)


0

In [86]:
import requests as req
import pandas as pd
import os
import glob
import subprocess
import zipfile
import  yfinance as yf
from typing import List
import beaapi as bea
from dotenv import load_dotenv
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

load_dotenv()


True

In [87]:
def fetch_and_process_etf_data(etf_list):
    ssga = [etf.lower() for etf in etf_list]
    subfolder = 'SSGA Data'
    if not os.path.exists(subfolder):
        os.makedirs(subfolder)

    ssga_df_list = []

    for etf in ssga:
        url = f'https://www.ssga.com/library-content/products/fund-data/etfs/us/holdings-daily-us-en-{etf}.xlsx'
        response = req.get(url)
        current_date = datetime.now().strftime('%m-%d-%Y')
        file_path = os.path.join(subfolder, f'{etf}-{current_date}.xlsx')
        with open(file_path, 'wb') as file:
            file.write(response.content)
        df = pd.read_excel(file_path, skiprows=4, header=0, usecols="A:H")
        
        drop_index = df[df['Name'] == "Past performance is not a reliable indicator of future performance. Investment return and principal value will fluctuate, so you may have a gain or loss when shares are sold. Current performance may be higher or lower than that quoted. All results are historical and assume the reinvestment of dividends and capital gains. Visit www.ssga.com for most recent month-end performance. "].index

        if not drop_index.empty:
            df = df[:drop_index[0]]

        if df.iloc[-1].isna().all():
            df = df[:-1]

        ssga_df_list.append(df)

    return ssga_df_list

# Testing Output
etf_list = ['XLI', 'XLK', 'XLE', 'XLB']
ssga_df_list = fetch_and_process_etf_data(etf_list)
for df in ssga_df_list:
    print(df.tail())

                             Name  Ticker Identifier    SEDOL    Weight  \
76           GENERAC HOLDINGS INC    GNRC  368736104  B6197Q2  0.195883   
77   HUNTINGTON INGALLS INDUSTRIE     HII  446413106  B40SSC9  0.185974   
78                      US DOLLAR       -  999USDZ92        -  0.025554   
79  SSI US GOV MONEY MARKET CLASS       -  924QSGII3        -  0.018963   
80        XAI EMINI INDUSTR MAR25  AIXH25  ADI2SGVK5        - -0.002176   

   Sector  Shares Held Local Currency  
76      -    303522.00            USD  
77      -    200836.00            USD  
78      -   5408113.03            USD  
79      -   4013194.45            USD  
80      -     25800.00            USD  
                             Name  Ticker Identifier    SEDOL    Weight  \
68             ENPHASE ENERGY INC    ENPH  29355A107  B65SQW4  0.095184   
69                      US DOLLAR       -  999USDZ92        -  0.038426   
70  SSI US GOV MONEY MARKET CLASS       -  924QSGII3        -  0.027552   
71       

In [88]:
def fetch_bea_data(api_key, years):
    base_url = "https://apps.bea.gov/api/data/"
    params = {
        "UserID": api_key,
        "method": "GetData",
        "datasetname": "GDPbyIndustry",
        "Frequency": "Q",
        "Year": years,
        "Industry": "ALL",
        "TableID": "ALL",
        "ResultFormat": "JSON"
    }
    try:
        response = req.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if 'Error' in data['BEAAPI']:
            error_code = data['BEAAPI']['Error']['APIErrorCode']
            error_description = data['BEAAPI']['Error']['APIErrorDescription']
            print(f"API request failed with error code {error_code}: {error_description}")
            return None
        return data
    except req.exceptions.RequestException as e:
        print(f"API request failed: {e}")
        return None

def process_bea_data(data):
    if 'BEAAPI' not in data or 'Results' not in data['BEAAPI']:
        print("Error: 'Results' key not found in the API response.")
        return None
    results = data['BEAAPI']['Results']
    df = pd.DataFrame(results)
    nested_data = df.loc[0, "Data"]
    bea_df = pd.DataFrame(nested_data)
    bea_df.rename(columns={"IndustrYDescription": "IndustryDescription"}, inplace=True)
    bea_df['DataValue'] = bea_df['DataValue'].astype(float)
    
    # Pivot the data to have years and quarters as columns
    bea_df['YearQuarter'] = bea_df['Year'].astype(str) + 'Q' + bea_df['Quarter']
    pivot_df = bea_df.pivot_table(index=['Industry', 'IndustryDescription'], columns='YearQuarter', values='DataValue', aggfunc='sum').reset_index()
    
    # Filter out unwanted industry descriptions
    filtered_df = pivot_df[~pivot_df['IndustryDescription'].isin([
        "Taxes on production and imports less subsidies",
        "Energy inputs",
        "Intermediate inputs",
        "Materials inputs",
        "Purchased-services inputs",
        "Value added",
        "Compensation of employees",
        "Gross operating surplus"
    ])]
    
    return filtered_df

def save_bea_data(filtered_df):
    subfolder = 'BEA Data'
    os.makedirs(subfolder, exist_ok=True)
    current_date = datetime.now().strftime('%m-%d-%Y')
    file_path = os.path.join(subfolder, f"bea-gdp-by-industry-raw-{current_date}.xlsx")
    filtered_df.to_excel(file_path, index=False)
    return file_path

def map_sectors(filtered_df):
    sector_map = {
        "Technology": [
            "Computer and electronic products",
            "Computer systems design and related services",
            "Data processing, internet publishing, and other information services",
            "Information-communications-technology-producing industries"
        ],
        "Materials": [
            "Agriculture, forestry, fishing, and hunting",
            "Farms",
            "Forestry, fishing, and related activities",
            "Mining",
            "Mining, except oil and gas",
            "Support activities for mining",
            "Wood products",
            "Paper products",
            "Chemical products",
            "Plastics and rubber products",
            "Nonmetallic mineral products",
            "Primary metals",
            "Fabricated metal products"
        ],
        "Energy": [
            "Oil and gas extraction",
            "Petroleum and coal products",
            "Pipeline transportation"
        ],
        "Industrials": [
            "Construction",
            "Machinery",
            "Electrical equipment, appliances, and components",
            "Other transportation equipment",
            "Miscellaneous manufacturing",
            "Durable goods",
            "Wholesale trade",
            "Rail transportation",
            "Water transportation",
            "Truck transportation",
            "Transit and ground passenger transportation",
            "Other transportation and support activities",
            "Transportation and warehousing",
            "Warehousing and storage",
            "Waste management and remediation services"
        ]
    }

    def get_sector(category):
        for sector, cat_list in sector_map.items():
            if category in cat_list:
                return sector
        return "Other"

    filtered_df = filtered_df.copy()
    filtered_df.loc[:, "Sector"] = filtered_df["IndustryDescription"].apply(get_sector)
    focus_sectors = ["Technology", "Materials", "Energy", "Industrials"]
    df_filtered = filtered_df[filtered_df["Sector"].isin(focus_sectors)]
    return df_filtered

def fetch_bea_gdp_by_industry(api_key, years):
    data = fetch_bea_data(api_key, years)
    if data is None:
        return None
    filtered_df = process_bea_data(data)
    if filtered_df is not None:
        save_bea_data(filtered_df)
        df_filtered = map_sectors(filtered_df)
        current_date = datetime.now().strftime('%m-%d-%Y')
        subfolder = 'BEA Data'
        filtered_file_path = os.path.join(subfolder, f"bea-gdp-by-industry-filtered-{current_date}.xlsx")
        df_filtered.to_excel(filtered_file_path, index=False)
        return df_filtered
    else:
        print("Error: Processed data is None.")
        return None

# Output the filtered DataFrame
api_key = os.environ.get("beakey")
years = "2020,2021,2022,2023, 2024"
bea_df = fetch_bea_gdp_by_industry(api_key, years)
if bea_df is not None:
    print(bea_df.tail())

YearQuarter Industry                                IndustryDescription  \
43              48TW                     Transportation and warehousing   
44               493                            Warehousing and storage   
50               514  Data processing, internet publishing, and othe...   
62              5415       Computer systems design and related services   
66               562          Waste management and remediation services   

YearQuarter    2020QI   2020QII  2020QIII   2020QIV    2021QI   2021QII  \
43           5959.009  4382.610  5399.989  5646.258  5795.131  6118.576   
44           1125.151  1037.465  1346.907  1269.619  1350.713  1274.375   
50           2790.249  2701.492  3137.504  3289.488  3402.355  3571.411   
62           2973.620  2856.763  2841.172  3072.813  3167.013  3281.216   
66           1166.495   993.554  1209.515  1198.336  1187.525  1187.900   

YearQuarter  2021QIII   2021QIV  ...  2022QIII   2022QIV    2023QI   2023QII  \
43           6429.

In [89]:
def setup_kaggle_data():
    # Step 1: Ensure the "Kaggle Data" directory exists
    kaggle_data_dir = "Kaggle Data"
    os.makedirs(kaggle_data_dir, exist_ok=True)
    
    # Step 2: Check if the .csv file already exists
    csv_exists = any(filename.endswith('.csv') for filename in os.listdir(kaggle_data_dir))
    
    if not csv_exists:
        # Step 3: Download the dataset using the Kaggle CLI
        dataset = "jakewright/9000-tickers-of-stock-market-data-full-history"
        subprocess.run(["kaggle", "datasets", "download", "-d", dataset], check=True)
        
        # Step 4: Unzip the downloaded file
        zip_filename = dataset.split('/')[-1] + ".zip"
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(kaggle_data_dir)
        
        # Step 5: Remove any files in the "Kaggle Data" directory that are not .csv files
        for filename in os.listdir(kaggle_data_dir):
            if not filename.endswith('.csv'):
                os.remove(os.path.join(kaggle_data_dir, filename))
        
        # Optionally, remove the downloaded zip file
        os.remove(zip_filename)

# Call the function
setup_kaggle_data()

# Load the CSV file into a DataFrame
kag_df = pd.read_csv('Kaggle Data/all_stock_data.csv')

# Convert the 'Date' column to datetime format
kag_df['Date'] = pd.to_datetime(kag_df['Date'])

# Filter the DataFrame for dates between 2020-01-01 and 2023-12-31
start_date = '2020-01-01'
end_date = '2023-12-31'
filtered_kag_df = kag_df[(kag_df['Date'] >= start_date) & (kag_df['Date'] <= end_date)]

# Display the first few rows of the filtered DataFrame
filtered_kag_df.head()

Dataset URL: https://www.kaggle.com/datasets/jakewright/9000-tickers-of-stock-market-data-full-history
License(s): other


100%|██████████| 1.79G/1.79G [00:28<00:00, 67.7MB/s]


,Date,Ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits
24919429,2020-01-02,HOFV,232.320007,232.539993,231.660004,231.660004,11459.0,0.0,0.0
24919430,2020-01-02,CTSH,61.055629,61.222259,60.457717,60.692959,2234500.0,0.0,0.0
24919431,2020-01-02,AZUL,43.610001,44.080002,43.150002,43.590000,532300.0,0.0,0.0
24919432,2020-01-02,FNCTF,12.707134,12.707134,12.707134,12.707134,100.0,0.0,0.0
24919433,2020-01-02,BLE,13.720488,13.720488,13.621064,13.657218,54700.0,0.0,0.0


In [90]:
filtered_kag_df.tail()

,Date,Ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits
32724160,2023-12-29,RTO,28.459999,28.620001,28.260000,28.610001,620800.0,0.0,0.0
32724161,2023-12-29,SHZNY,41.020000,41.020000,41.020000,41.020000,0.0,0.0,0.0
32724162,2023-12-29,ONMD,0.745000,0.900000,0.745000,0.810000,70100.0,0.0,0.0
32724163,2023-12-29,KMPR,48.349998,49.150002,48.150002,48.669998,312300.0,0.0,0.0
32724164,2023-12-29,SBT,5.850000,5.870000,5.770000,5.770000,14200.0,0.0,0.0


In [91]:
def ensure_directory_exists(directory: str) -> None:
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' created.")
    else:
        print(f"Directory '{directory}' already exists.")

def download_data(tickers: List[str], start_date: str, end_date: str, interval: str) -> pd.DataFrame:
    return yf.download(
        tickers=tickers,
        start=start_date,
        end=end_date,
        interval=interval,
        group_by="ticker",
        auto_adjust=True,
        threads=True,
        progress=True
    )

def filter_columns(df: pd.DataFrame, tickers: List[str], fields: List[str]) -> pd.DataFrame:
    keep_cols = [(t, field) for t in tickers for field in fields if (t, field) in df.columns]
    return df[keep_cols].copy()

def add_change_columns(df: pd.DataFrame, tickers: List[str]) -> pd.DataFrame:
    for t in tickers:
        if (t, "High") in df.columns and (t, "Low") in df.columns:
            df[(t, "Change")] = df[(t, "High")] - df[(t, "Low")]
    return df

def process_change_columns(df: pd.DataFrame) -> pd.DataFrame:
    df_only_change = df.loc[:, (slice(None), "Change")].copy()
    df_only_change.columns = df_only_change.columns.droplevel(1)
    df_only_change.columns = [f"{ticker}_Change" for ticker in df_only_change.columns]
    df_only_change.reset_index(inplace=True)
    if df_only_change.columns[0] == "index":
        df_only_change.rename(columns={"index": "Date"}, inplace=True)
    return df_only_change

def save_to_csv(df: pd.DataFrame, path: str) -> None:
    df.to_csv(path, index=False)
    print(f"Data with only 'Change' columns saved to: {path}")


# Main execution flow
data_dir = "Yahoo Data"
ensure_directory_exists(data_dir)

tickers = ["XLI", "XLK", "XLE", "XLB"]
df_full = download_data(tickers, "2020-01-01", "2024-01-01", "3mo")

df_filtered = filter_columns(df_full, tickers, ["High", "Low"])
df_filtered = add_change_columns(df_filtered, tickers)

df_only_change = process_change_columns(df_filtered)

csv_path = os.path.join(data_dir, "sector_quarterly_only_change.csv")
save_to_csv(df_only_change, csv_path)

print(df_only_change.head())

[*********************100%***********************]  4 of 4 completed

Directory 'Yahoo Data' created.
Data with only 'Change' columns saved to: Yahoo Data/sector_quarterly_only_change.csv
        Date  XLI_Change  XLK_Change  XLE_Change  XLB_Change
0 2020-01-01   34.623135   33.295942   30.822577   21.908651
1 2020-04-01   19.467889   28.695256   16.112842   16.252516
2 2020-07-01   12.474146   23.360491    8.296762   10.461685
3 2020-10-01   14.514662   21.075830   12.746975   10.079117
4 2021-01-01   13.851598   14.060702   14.423070    9.381535


In [92]:
# Step 1: Read the SSGA Excel files and extract the top 10 stocks by weight
ssga_files = glob.glob('/home/pinagm/dev/Udacity_DataAnalyst_NanoDegree/D502 - Capstone Project/SSGA Data/*.xlsx')
top_tickers = []

for file in ssga_files:
    df = pd.read_excel(file, skiprows=4)  # Adjust skiprows if necessary
    top_10 = df.nlargest(10, 'Weight')['Ticker'].tolist()
    top_tickers.extend(top_10)

# Remove duplicates
top_tickers = list(set(top_tickers))

# Step 2: Filter the existing filtered_kag_df based on the extracted tickers
filtered_kag_df = filtered_kag_df[filtered_kag_df['Ticker'].isin(top_tickers)]

# Step 3: Filter the DataFrame for dates between 2020-01-01 and 2023-12-31
start_date = '2020-01-01'
end_date = '2023-12-31'
filtered_kag_df = filtered_kag_df[(filtered_kag_df['Date'] >= start_date) & (filtered_kag_df['Date'] <= end_date)]

# Step 4: Convert the 'Date' column to datetime format
filtered_kag_df['Date'] = pd.to_datetime(filtered_kag_df['Date'])

# Step 5: Set the 'Date' column as the index
filtered_kag_df.set_index('Date', inplace=True)

# Step 6: Resample the data to a quarterly frequency and aggregate the values
quarterly_kag_df = filtered_kag_df.groupby('Ticker').resample('Q').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum',
    'Dividends': 'sum',
    'Stock Splits': 'sum'
}).reset_index()

# Display the first few rows of the quarterly DataFrame
quarterly_kag_df.head()

# Save the quarterly consolidated dataset to a new CSV file
quarterly_kag_df.to_csv('/home/pinagm/dev/Udacity_DataAnalyst_NanoDegree/D502 - Capstone Project/Kaggle Data/filtered_stock_data_quarterly.csv', index=False)

In [93]:
bea_excel = 'BEA Data/bea-gdp-by-industry-filtered-03-10-2025.xlsx'

if os.path.exists(bea_excel):
	bea_excel_df = pd.read_excel(bea_excel)
	print(bea_excel_df.head())
else:
	print(f"File not found: {bea_excel}")
 
kag_csv = 'Kaggle Data/filtered_stock_data_quarterly.csv'

if os.path.exists(kag_csv):
    kag_csv_df = pd.read_csv(kag_csv)
    print(kag_csv_df.head())
else:
    print(f"File not found: {kag_csv}")



  Industry                          IndustryDescription    2020QI   2020QII  \
0       11  Agriculture, forestry, fishing, and hunting  2441.610  2126.918   
1    111CA                                        Farms  2211.313  1909.135   
2    113FF    Forestry, fishing, and related activities   781.368   759.700   
3       21                                       Mining  2650.141  1637.385   
4      211                       Oil and gas extraction  1697.653  1088.254   

   2020QIII   2020QIV    2021QI   2021QII  2021QIII   2021QIV  ...  2022QIII  \
0  2535.020  2592.121  2568.646  2840.182  2732.267  2683.640  ...  2962.575   
1  2348.416  2360.465  2368.935  2637.372  2495.972  2444.318  ...  2724.594   
2   857.438   991.167  1038.892   943.554   927.734   861.779  ...   963.665   
3  2495.354  2493.640  2874.500  3151.607  3092.183  3554.839  ...  3930.475   
4  3174.931  2061.840  2511.923  2765.309  2549.100  3140.963  ...  3300.296   

    2022QIV    2023QI   2023QII  2023QIII   

In [131]:
import pandas as pd

# Suppose your original wide BEA DataFrame is called bea_df with columns:
# ['Industry','IndustryDescription','2020QI','2020QII','2020QIII',...,'Sector']

# 1) Identify columns that look like e.g., "2020QI", "2020QII", ...
quarter_cols = [
    c for c in bea_excel_df.columns
    if c not in ['Industry','IndustryDescription','Sector']
    and 'Q' in c
]

# 2) Melt the DataFrame
bea_melted = bea_excel_df.melt(
    id_vars=['Industry','IndustryDescription','Sector'],
    value_vars=quarter_cols,
    var_name='QuarterStr',   # e.g., "2020QI", "2021QIV"
    value_name='GDP'         # the numeric GDP value
)

print(bea_melted.head(10))


  Industry                          IndustryDescription       Sector  \
0       11  Agriculture, forestry, fishing, and hunting    Materials   
1    111CA                                        Farms    Materials   
2    113FF    Forestry, fishing, and related activities    Materials   
3       21                                       Mining    Materials   
4      211                       Oil and gas extraction       Energy   
5      212                   Mining, except oil and gas    Materials   
6      213                Support activities for mining    Materials   
7       23                                 Construction  Industrials   
8      321                                Wood products    Materials   
9      322                               Paper products    Materials   

  QuarterStr       GDP  
0     2020QI  2441.610  
1     2020QI  2211.313  
2     2020QI   781.368  
3     2020QI  2650.141  
4     2020QI  1697.653  
5     2020QI   962.935  
6     2020QI  1219.103  
7     2

In [132]:
import re

roman_map = {'I': '1', 'II': '2', 'III': '3', 'IV': '4'}

def parse_quarter_str(qstr):
    """
    Convert strings like '2020QI' or '2021QIII' into the end-of-quarter date:
    e.g., '2020-03-31'.
    """
    match = re.match(r"^(\d{4})Q(I|II|III|IV)$", qstr)
    if match:
        year_str = match.group(1)           # e.g. '2020'
        roman_quarter = match.group(2)      # e.g. 'II'
        year = int(year_str)
        quarter_num = int(roman_map[roman_quarter])  # 1..4
        period = pd.Period(year=year, quarter=quarter_num, freq='Q')
        return period.end_time  # e.g. Timestamp('2020-06-30 00:00:00')
    else:
        return None  # If doesn't match pattern
    
bea_melted['QuarterEnd'] = bea_melted['QuarterStr'].apply(parse_quarter_str)
bea_melted.dropna(subset=['QuarterEnd'], inplace=True)
bea_melted.drop(columns=['QuarterStr'], inplace=True)


print(bea_melted.head(10))


  Industry                          IndustryDescription       Sector  \
0       11  Agriculture, forestry, fishing, and hunting    Materials   
1    111CA                                        Farms    Materials   
2    113FF    Forestry, fishing, and related activities    Materials   
3       21                                       Mining    Materials   
4      211                       Oil and gas extraction       Energy   
5      212                   Mining, except oil and gas    Materials   
6      213                Support activities for mining    Materials   
7       23                                 Construction  Industrials   
8      321                                Wood products    Materials   
9      322                               Paper products    Materials   

        GDP                    QuarterEnd  
0  2441.610 2020-03-31 23:59:59.999999999  
1  2211.313 2020-03-31 23:59:59.999999999  
2   781.368 2020-03-31 23:59:59.999999999  
3  2650.141 2020-03-31 23:59:59

In [133]:
bea_melted['QuarterEnd'] = (
    bea_melted['QuarterEnd'].dt.to_period('Q')
    .dt.to_timestamp(freq='Q')
)

In [134]:
print(bea_melted.head(10))

  Industry                          IndustryDescription       Sector  \
0       11  Agriculture, forestry, fishing, and hunting    Materials   
1    111CA                                        Farms    Materials   
2    113FF    Forestry, fishing, and related activities    Materials   
3       21                                       Mining    Materials   
4      211                       Oil and gas extraction       Energy   
5      212                   Mining, except oil and gas    Materials   
6      213                Support activities for mining    Materials   
7       23                                 Construction  Industrials   
8      321                                Wood products    Materials   
9      322                               Paper products    Materials   

        GDP QuarterEnd  
0  2441.610 2020-03-31  
1  2211.313 2020-03-31  
2   781.368 2020-03-31  
3  2650.141 2020-03-31  
4  1697.653 2020-03-31  
5   962.935 2020-03-31  
6  1219.103 2020-03-31  
7  7632

In [135]:
kag_csv_df['Date'] = pd.to_datetime(kag_csv_df['Date'])

kag_csv_df['QuarterEnd'] = (
    kag_csv_df['Date'].dt.to_period('Q')
    .dt.to_timestamp(freq='Q')
)

print(kag_csv_df.head(10))

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2020-03-31   73.192005   81.038115   52.654220   62.976532   
1   AAPL 2020-06-30   61.047293   92.471916   58.669790   90.589600   
2   AAPL 2020-09-30   90.669063  137.303621   88.548348  115.242294   
3   AAPL 2020-12-31  117.063323  138.347911  106.793920  132.267349   
4   AAPL 2021-03-31  133.094712  144.627854  116.012942  121.942871   
5   AAPL 2021-06-30  123.450312  137.410004  122.250000  136.960007   
6   AAPL 2021-09-30  136.600006  157.259995  135.759995  141.500000   
7   AAPL 2021-12-31  141.899994  182.130005  138.270004  177.570007   
8   AAPL 2022-03-31  177.830002  182.940002  150.100006  174.610001   
9   AAPL 2022-06-30  174.029999  178.490005  129.039993  136.720001   

         Volume  Dividends  Stock Splits QuarterEnd  Quarterly_Return  
0  1.223372e+10     0.1925           0.0 2020-03-31               NaN  
1  9.314611e+09     0.2050           0.0 2020-06-30          0.438466  
2 

In [137]:
etf_to_sector = {
    'XLI': 'Industrials',
    'XLK': 'Technology',
    'XLE': 'Energy',
    'XLB': 'Materials'
}
# 1. Create an empty list to store the updated DataFrames
updated_ssga_dfs = []

# 2. Loop over each DataFrame in ssga_df_list
for etf_name, df in zip(etf_list, ssga_df_list):
    # Optionally rename columns to avoid confusion
    df.rename(columns={'Ticker': 'HoldingTicker'}, inplace=True)
    
    # Add columns for ETF ticker & the correct sector
    df['ETF_Ticker'] = etf_name
    df['Sector'] = etf_to_sector[etf_name]
    
    updated_ssga_dfs.append(df)

# 3. Concatenate all the updated DataFrames into one
combined_ssga_df = pd.concat(updated_ssga_dfs, ignore_index=True)

# 4. Inspect the final combined DataFrame
print(combined_ssga_df.head(10))
print(combined_ssga_df['ETF_Ticker'].unique())
print(combined_ssga_df['Sector'].unique())


                          Name HoldingTicker Identifier    SEDOL    Weight  \
0             GENERAL ELECTRIC            GE  369604301  BL59CR9  5.062485   
1                     RTX CORP           RTX  75513E101  BM5M5Y3  4.131536   
2              CATERPILLAR INC           CAT  149123101  2180201  4.088638   
3        UBER TECHNOLOGIES INC          UBER  90353T100  BK6N347  3.882580   
4           UNION PACIFIC CORP           UNP  907818108  2914734  3.653992   
5  HONEYWELL INTERNATIONAL INC           HON  438516106  2020459  3.372322   
6                   DEERE + CO            DE  244199105  2261203  3.073441   
7    AUTOMATIC DATA PROCESSING           ADP  053015103  2065308  3.018781   
8                BOEING CO/THE            BA  097023105  2108601  2.786592   
9               EATON CORP PLC           ETN  G29183103  B8KQN82  2.712898   

        Sector  Shares Held Local Currency ETF_Ticker  
0  Industrials    5537135.0            USD        XLI  
1  Industrials    6809676.0  

In [139]:
xli_holdings = combined_ssga_df[combined_ssga_df['ETF_Ticker'] == 'XLI']
print(xli_holdings.head(10))

                          Name HoldingTicker Identifier    SEDOL    Weight  \
0             GENERAL ELECTRIC            GE  369604301  BL59CR9  5.062485   
1                     RTX CORP           RTX  75513E101  BM5M5Y3  4.131536   
2              CATERPILLAR INC           CAT  149123101  2180201  4.088638   
3        UBER TECHNOLOGIES INC          UBER  90353T100  BK6N347  3.882580   
4           UNION PACIFIC CORP           UNP  907818108  2914734  3.653992   
5  HONEYWELL INTERNATIONAL INC           HON  438516106  2020459  3.372322   
6                   DEERE + CO            DE  244199105  2261203  3.073441   
7    AUTOMATIC DATA PROCESSING           ADP  053015103  2065308  3.018781   
8                BOEING CO/THE            BA  097023105  2108601  2.786592   
9               EATON CORP PLC           ETN  G29183103  B8KQN82  2.712898   

        Sector  Shares Held Local Currency ETF_Ticker  
0  Industrials    5537135.0            USD        XLI  
1  Industrials    6809676.0  

In [141]:
# Merge Kaggle DF with combined SSGA DF
merged_kag_ssga = pd.merge(
    kag_csv_df,  # from Kaggle
    combined_ssga_df, 
    left_on='Ticker',     # from Kaggle
    right_on='HoldingTicker',  # from SSGA
    how='left'           # keep all Kaggle rows, even if no match
)

print("After merging Kaggle + SSGA, shape:", merged_kag_ssga.shape)
print(merged_kag_ssga.head())


After merging Kaggle + SSGA, shape: (640, 20)
  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2020-03-31   73.192005   81.038115   52.654220   62.976532   
1   AAPL 2020-06-30   61.047293   92.471916   58.669790   90.589600   
2   AAPL 2020-09-30   90.669063  137.303621   88.548348  115.242294   
3   AAPL 2020-12-31  117.063323  138.347911  106.793920  132.267349   
4   AAPL 2021-03-31  133.094712  144.627854  116.012942  121.942871   

         Volume  Dividends  Stock Splits QuarterEnd  Quarterly_Return  \
0  1.223372e+10     0.1925           0.0 2020-03-31               NaN   
1  9.314611e+09     0.2050           0.0 2020-06-30          0.438466   
2  1.097559e+10     0.2050           4.0 2020-09-30          0.272136   
3  7.339933e+09     0.2050           0.0 2020-12-31          0.147733   
4  6.724536e+09     0.2050           0.0 2021-03-31         -0.078058   

        Name HoldingTicker Identifier    SEDOL     Weight      Sector  \
0  APPLE INC   

In [144]:
bea_sector_agg = (
    bea_melted
    .groupby(["Sector","QuarterEnd"], as_index=False)
    .agg({"GDP": "sum"})  # You might choose mean, sum, or something else
)

final_merged_df = pd.merge(
    merged_kag_ssga,
    bea_sector_agg,
    on=["Sector","QuarterEnd"],
    how="inner"
)

print("Final shape:", final_merged_df.shape)
print(final_merged_df.tail())


Final shape: (640, 21)
    Ticker       Date        Open        High         Low       Close  \
635    OKE 2023-12-31   63.680000   71.440002   60.580002   70.220001   
636    PSX 2023-12-31  120.089996  136.690002  107.849998  133.139999   
637    SLB 2023-12-31   58.320000   61.200001   48.090000   52.040001   
638    WMB 2023-12-31   33.540001   37.450001   32.490002   34.830002   
639    XOM 2023-12-31  117.529999  117.790001   97.480003   99.980003   

           Volume  Dividends  Stock Splits QuarterEnd  ...              Name  \
635  2.510922e+08        0.0           0.0 2023-12-31  ...         ONEOK INC   
636  1.969818e+08        0.0           0.0 2023-12-31  ...       PHILLIPS 66   
637  5.427724e+08        0.0           0.0 2023-12-31  ...  SCHLUMBERGER LTD   
638  4.054222e+08        0.0           0.0 2023-12-31  ...  WILLIAMS COS INC   
639  1.381208e+09        0.0           0.0 2023-12-31  ...  EXXON MOBIL CORP   

    HoldingTicker Identifier    SEDOL     Weight  Sector 